In [1]:
import numpy as np
import random
random.seed(7)

In [2]:
string = "0ABCD0"
vocab = {'A':0,'B':1,'C':2,'D':3, '0':4}
states = 3
symbols = 5

### Transition and Emission Probabilities

In [3]:
#Generate random transition and emission probabilities
A = [[0]+[random.random() for i in range(states)]+[0]]
B = [[0 for i in range(symbols)]]
for i in range(states):
    transition_probabilities = [0]+[random.random() for i in range(states+1)]
    emission_probabilities = [random.random() for i in range(symbols)]
    B.append(emission_probabilities)
    A.append(transition_probabilities)
A.append([0 for i in range(states+1)]+[1])
B.append([0 for i in range(symbols)])
B[0][-1] = 1
B[-1][-1] = 1
A = np.array(A)
B = np.array(B)

#normalize A and B
for i in range(len(A)):
    s = sum(A[i])
    A[i] = A[i]/s
    s = sum(B[i])
    if s!=0:
        B[i] = B[i]/s
        
print("Transition Probabilities:")
print(A)
print("Emission Probabilities:")
print(B)

Transition Probabilities:
[[0.         0.28769371 0.13401473 0.57829156 0.        ]
 [0.         0.07018978 0.51926242 0.35434762 0.05620017]
 [0.         0.26558805 0.51729592 0.07745309 0.13966295]
 [0.         0.03479135 0.64116701 0.21630139 0.10774024]
 [0.         0.         0.         0.         1.        ]]
Emission Probabilities:
[[0.         0.         0.         0.         1.        ]
 [0.44545295 0.0329156  0.38067629 0.06132265 0.07963251]
 [0.17798612 0.26883982 0.16370876 0.1125277  0.27693761]
 [0.05875725 0.15387723 0.407101   0.09015012 0.2901144 ]
 [0.         0.         0.         0.         1.        ]]


### Forward Procedure

In [4]:
def forward_procedure(string):   
    length = len(string)

    first_char = vocab[string[0]]
    alpha = np.zeros((length,states+2))

    alpha[0,:] = [ A[0][i]*B[i][ first_char ] for i in range(0,states+2) ]
    for t in range(1,length):
        alpha_t = []
        for j in range(0, states+2):
            s = 0
            for i in range(0,states+2):
                s += alpha[t-1][i]*A[i][j]
            alpha[t,j] = s*B[j][vocab[string[t]]]
    return alpha

alpha = forward_procedure('0ABCD0')
print(alpha)

[[0.00000000e+00 2.29097717e-02 3.71137173e-02 1.67770710e-01
  0.00000000e+00]
 [0.00000000e+00 7.70720986e-03 2.46802675e-02 2.77813860e-03
  0.00000000e+00]
 [0.00000000e+00 2.36742384e-04 4.98706385e-03 8.06856711e-04
  0.00000000e+00]
 [0.00000000e+00 5.21219343e-04 5.27150234e-04 2.62448579e-04
  0.00000000e+00]
 [0.00000000e+00 1.13888439e-05 8.00764726e-05 2.54484994e-05
  0.00000000e+00]
 [0.00000000e+00 1.82773505e-06 1.76281225e-05 4.56707533e-06
  1.45655987e-05]]


### Backward Procedure

In [5]:
def backward_procedure(string):
    length = len(string)

    beta = np.zeros((length,states+2))

    beta[length-1,:] = [ 1 for i in range(states+2)]

    for t in range(length-2,-1,-1):
        for i in range(0, states+2):
            s = 0
            for j in range(0, states+2):
                s += A[i][j]*B[j][vocab[string[t+1]]]*beta[t+1,j]
            beta[t,i] = s
    return beta

beta = backward_procedure('0ABCD0')
print(beta)

[[2.27676200e-04 1.61150960e-04 2.47104920e-04 1.53337956e-04
  0.00000000e+00]
 [8.30943347e-04 1.23697750e-03 1.02951452e-03 1.31245889e-03
  0.00000000e+00]
 [1.13552429e-02 7.57612526e-03 6.41815848e-03 5.93311738e-03
  0.00000000e+00]
 [2.86547380e-02 3.16146272e-02 2.64803026e-02 3.10585744e-02
  0.00000000e+00]
 [2.27794199e-01 3.08394203e-01 3.26541339e-01 3.50826173e-01
  1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00]]


### Xi t,i,j
Probability of being in state i at time t and moving to state j, given a set of observations O0 to OT

In [6]:
def Xi(t,i,j,string):
    num = alpha[t,i] * A[i][j] * B[j][vocab[string[t+1]]] * beta[t+1][j]
    den = 0
    for ti in range(0,states+2):
        for tj in range(0,states+2):
            den += alpha[t,ti] * A[ti][tj] * B[tj][vocab[string[t+1]]] * beta[t+1][tj]
    return num/den

### Gamma t,i

In [7]:
def gamma(t,i,string):
    g = 0
    for j in range(0,states+2):
        g += Xi(t,i,j,string)
    return g
gamma(0,1,string)+gamma(0,2,string)+gamma(0,3,string)

1.0

In [8]:
def get_pi(string):
    pi = []
    for i in range(0,states+2):
        pi.append(gamma(0,i,string))
    return pi

In [9]:
def get_A(string):
    new_A = np.zeros((states+2, states+2))
    for i in range(0, states+2):
        for j in range(0, states+2):
            num = 0
            den = 0
            T = len(string)
            for t in range(0, T-1):
                num += Xi(t,i,j,string)
                den += gamma(t,i,string)
            if num:
                new_A[i][j] = num/den
            else:
                new_A[i][j] = 0
    new_A[-1,-1] = 1
    return new_A

In [10]:
def get_B(string):
    T = len(string)
    new_B = np.zeros((states+2,symbols))
    for j in range(0,states+2):
        for k in range(symbols):
            num = 0
            den = 0
            for t in range(0,T-1):
                if vocab[string[t]] == k:
                    num += gamma(t,j,string)
                den += gamma(t,j,string)
            if num:
                new_B[j][k] = num/den
            else:
                new_B[j][k] = 0
    new_B[0][vocab['0']] = 1
    new_B[-1][vocab['0']] = 1
    return new_B

In [11]:
nPi = get_pi("0ABCD0")
nA = get_A("0ABCD0")
nB = get_B("0ABCD0")
nA[0] = nPi
print("New Transition Matrix:\n",nA)
print("New Emission Matrix:\n",nB)
# get_B("0ABCD0")

New Transition Matrix:
 [[0.         0.09567432 0.23766082 0.66666485 0.        ]
 [0.         0.05645478 0.60557462 0.31968832 0.01828228]
 [0.         0.25631546 0.54568847 0.09317639 0.10481968]
 [0.         0.07458077 0.69730925 0.17459865 0.05351133]
 [0.         0.         0.         0.         1.        ]]
New Emission Matrix:
 [[0.         0.         0.         0.         1.        ]
 [0.27231536 0.05123141 0.47067566 0.10032265 0.10545491]
 [0.23814365 0.29999363 0.13083201 0.24507543 0.08595527]
 [0.07116152 0.0934297  0.15908577 0.17424477 0.50207824]
 [0.         0.         0.         0.         1.        ]]


In [22]:
def prob(string):
    alpha = forward_procedure(string)
    length = len(string)
    s = 0
    for i in range(0,states+2):
        s += alpha[length-1][i]
    return s
prob('0AAA0')

0.00039031428207478964